In [ ]:
import torch

from model.resnet_model import CNNClassifier
from torch.utils.data import DataLoader
from pathlib import Path
from ml_utils import CustomDataset

from sklearn.metrics import f1_score, accuracy_score
from torch.nn import CrossEntropyLoss

In [ ]:
device = "mps"

In [ ]:
test_dt = CustomDataset(data_path=Path("../datasets/car_dataset"), portion="test")
test_loader = DataLoader(test_dt, batch_size=64, shuffle=True, num_workers=4)

cls = CNNClassifier(num_classes=196, pretrained=True, model_name="resnet18", freeze_layers=0)
cls.load_state_dict(torch.load("../experiments/results/r18_pretrained_fre0.pth", weights_only=True))
cls.eval()
cls = cls.to(device)

In [ ]:
from tqdm import tqdm

preds, trues = [], []
criterion = CrossEntropyLoss().to(device)

for idx, (x, y) in enumerate(tqdm(test_loader), 1):
    x, y = x.to(device), y.to(device)
    x.requires_grad_(True)

    loss = criterion(cls(x), y)
    cls.zero_grad()
    loss.backward()

    with torch.no_grad():
        adv_logits = cls(x + .007 * torch.sign(x.grad))

    predictions = torch.argmax(adv_logits, dim=1)
    preds.append(predictions)
    trues.append(y)

# Concatenate all predictions, true labels, and losses
preds = torch.cat(preds).cpu().numpy()
trues = torch.cat(trues).cpu().numpy()

f1_mean = f1_score(trues, preds, average='weighted')
acc_mean = accuracy_score(trues, preds)

In [ ]:
f1_mean

In [ ]:
acc_mean